In [122]:
import datetime
import re
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt

df2 = pd.read_csv('df2.csv')
df3 = pd.read_csv('df3.csv')
df4 = pd.read_csv('df4.csv')
df5 = pd.read_csv('df5.csv')
df6 = pd.read_csv('df6.csv')
df7 = pd.read_csv('df7.csv')
df8 = pd.read_csv('df8.csv')
df9 = pd.read_csv('df9.csv')
df10 = pd.read_csv('df10.csv')
df11 = pd.read_csv('df11.csv')


# remove columns if they miss in one of the dataframes
data = pd.concat([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11])
# data.drop(columns = set(df.columns) ^ set(df1.columns), inplace=True)
data = data[data['price']<60000]
data = data[data['price']>1000]


# modify date column to store length of time period since particular offer has been created
def remove_polish_lang_from_date(date_val):
    months =  {
        'stycznia': '1', 'lutego': '2', 'marca': '3',
        'kwietnia': '4', 'maja': '5', 'czerwca': '6',
        'lipca': '7', 'sierpnia': '8', 'września': '9',
        'października': '10', 'listopada': '11', 'grudnia': '12'
    }
    date_val = re.sub(r'^.*?,', '', date_val)
    for k, v in months.items():
        date_val = date_val.replace(k, v)
    return date_val

data['offer date'] = data['offer date'].apply(remove_polish_lang_from_date)
data['offer date'] =  (datetime.datetime.now() - pd.to_datetime(data['offer date'], format=' %d %m %Y')).dt.days


# merge generation along with car model to avoid name conflicts
data['Wersja'] += ' ' + data['Marka pojazdu']
data['Model pojazdu'] += ' ' + data['Generacja']

data.drop(columns=['Generacja'], inplace=True)

important_columns = {'offer date', 'Marka pojazdu', 'Model pojazdu', 'Rok produkcji', 'Przebieg',
                     'Pojemność skokowa', 'Rodzaj paliwa', 'Napęd', 'Typ', 'Skrzynia biegów',
                     'Bezwypadkowy', 'Liczba drzwi', 'Alufelgi', 'Wersja', 'Dach panoramiczny',
                     'Kierownica po prawej (Anglik)', 'Moc', 'price'}

data = data[important_columns]
data.drop_duplicates(inplace=True)

data['Kierownica po prawej (Anglik)'].fillna('Nie', inplace=True)
data['Napęd'].fillna('Na przednie koła', inplace=True)
data['Bezwypadkowy'].fillna('Tak', inplace=True)
data['Liczba drzwi'].fillna(4, inplace=True)
data['Alufelgi'].fillna(False, inplace=True)
data['Dach panoramiczny'].fillna(False, inplace=True)

data.dropna(subset=important_columns, inplace=True)

data['Przebieg'] = data['Przebieg'].str.replace(' km', '').str.replace(' ', '')
data['Pojemność skokowa'] = data['Pojemność skokowa'].str.replace(' cm3', '').str.replace(' ', '')
data['Moc'] = data['Moc'].str.replace(' KM', '').str.replace(' ', '')
for col in ['Rok produkcji', 'Przebieg', 'Pojemność skokowa', 'Moc', 'Liczba drzwi']:
    data[col] = pd.to_numeric(data[col])

numeric_columns = []
numeric_columns.extend(list(data.dtypes[data.dtypes == np.int64].index))
numeric_columns.extend(list(data.dtypes[data.dtypes == np.float64].index))
numeric_columns = set(numeric_columns) & important_columns
non_numeric_columns = important_columns - numeric_columns

numeric_data = data[numeric_columns]
non_numeric_data = data[non_numeric_columns]

from sklearn.preprocessing import OneHotEncoder
oh_encoder = OneHotEncoder(sparse=False)

X = non_numeric_data.copy()
y = numeric_data['price']
out = oh_encoder.fit_transform(X, y)
non_numeric_data = DataFrame(out, columns=oh_encoder.get_feature_names())
full_data = pd.merge(numeric_data, non_numeric_data, left_index=True, right_index=True)

############
# lista = []
# with open('lista.txt', 'r') as f:
#     for item in full_data.columns:
#         lista.append(f.readline())
#
# with open('lista2.txt', 'w') as f:
#     for item in set(lista) - set(full_data.columns):
#         f.write(f'{item}')
# full_data.columns

/usr/local/anaconda3/envs/umap_test/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (118,123) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/anaconda3/envs/umap_test/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (121) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/anaconda3/envs/umap_test/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (88,107,116,117,121,123) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [123]:
from sklearn.linear_model import LinearRegression, BayesianRidge, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
import umap
from sklearn import metrics
from sklearn.model_selection import cross_val_score

full_data = full_data.reset_index()

In [ ]:
temp_X = full_data.drop(columns=['price'])
y = full_data['price']

reduced_datas = {}
temp_X = StandardScaler().fit_transform(temp_X)
#################

# PCA
reduced_datas['pca']= DataFrame(data=PCA(n_components=20).fit_transform(temp_X))
# UMAP
reduced_datas['umap']= DataFrame(data=umap.UMAP(n_components=20).fit_transform(temp_X))
# SVD
reduced_datas['svd']= DataFrame(data=TruncatedSVD(n_components=20).fit_transform(temp_X))
# T-SNE
reduced_datas['tsne']= DataFrame(data=TSNE(n_components=2).fit_transform(temp_X))
################

In [ ]:
for reducer_name, X in reduced_datas.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    fig, axes = plt.subplots(1, 2)
    for i, model in enumerate([
                  # MLPRegressor(max_iter=10,learning_rate_init=0.001, hidden_layer_sizes=(100, 500, 200)),
                  LinearRegression(),
                  # BayesianRidge(),
                  # ElasticNet(),
                  # GradientBoostingRegressor(),
                  # ExtraTreesRegressor(),
                  # RandomForestRegressor(),
                  # BaggingRegressor(),
                  # AdaBoostRegressor()
                  ]):
        model.fit(X_train, y_train)
        print(f'{reducer_name} {model} MAE mean (cv=2) ={cross_val_score(model, X_test, y_test, scoring="neg_mean_absolute_error", cv=2).mean()}')
        y_pred = model.predict(X_test)

        fig.suptitle(f'Data after {reducer_name}')
        axes[0].scatter(y_test, y_pred)
        axes[0].set_title(f'{model}')
        axes[0].set(xlabel='real price', ylabel='predicted price', xlim=0, ylim=0)

        axes.flat[1].scatter(X[0], X[1])
        axes.flat[1].set_title('features dependency')
        axes.flat[1].set(xlabel='feature 1', ylabel='feature 2')

        plt.tight_layout()
        fig.savefig(f'{reducer_name}.jpg', dpi=1200)

        for ax in axes:
            ax.label_outer()

In [ ]:
for reducer_name, X in reduced_datas.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    fig, axes = plt.subplots(1, 2)
    for i, model in enumerate([
                  # MLPRegressor(max_iter=10,learning_rate_init=0.001, hidden_layer_sizes=(100, 500, 200)),
                  LinearRegression(),
                  # BayesianRidge(),
                  # ElasticNet(),
                  # GradientBoostingRegressor(),
                  # ExtraTreesRegressor(),
                  # RandomForestRegressor(),
                  # BaggingRegressor(),
                  # AdaBoostRegressor()
                  ]):
        model.fit(X_train, y_train)
        print(f'{reducer_name} {model} MAE mean (cv=2) ={cross_val_score(model, X_test, y_test, scoring="neg_mean_absolute_error", cv=2).mean()}')
        y_pred = model.predict(X_test)

        fig.suptitle(f'Data after {reducer_name}')
        axes[0].scatter(y_test, y_pred)
        axes[0].set_title(f'{model}')
        axes[0].set(xlabel='real price', ylabel='predicted price', xlim=0, ylim=0)

        axes.flat[1].scatter(X[0], X[1])
        axes.flat[1].set_title('features dependency')
        axes.flat[1].set(xlabel='feature 1', ylabel='feature 2')

        plt.tight_layout()
        fig.savefig(f'{reducer_name}.jpg', dpi=1200)

        for ax in axes:
            ax.label_outer()

/usr/local/anaconda3/envs/umap_test/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [ ]:
for reducer_name, X in reduced_datas.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    fig, axes = plt.subplots(1, 2)
    for i, model in enumerate([
                  # MLPRegressor(max_iter=10,learning_rate_init=0.001, hidden_layer_sizes=(100, 500, 200)),
                  LinearRegression(),
                  # BayesianRidge(),
                  # ElasticNet(),
                  # GradientBoostingRegressor(),
                  # ExtraTreesRegressor(),
                  # RandomForestRegressor(),
                  # BaggingRegressor(),
                  # AdaBoostRegressor()
                  ]):
        model.fit(X_train, y_train)
        print(f'{reducer_name} {model} MAE mean (cv=2) ={cross_val_score(model, X_test, y_test, scoring="neg_mean_absolute_error", cv=2).mean()}')
        y_pred = model.predict(X_test)

        fig.suptitle(f'Data after {reducer_name}')
        axes[0].scatter(y_test, y_pred)
        axes[0].set_title(f'{model}')
        axes[0].set(xlabel='real price', ylabel='predicted price', xlim=0, ylim=0)

        axes.flat[1].scatter(X[0], X[1])
        axes.flat[1].set_title('features dependency')
        axes.flat[1].set(xlabel='feature 1', ylabel='feature 2')

        plt.tight_layout()
        fig.savefig(f'{reducer_name}.jpg', dpi=1200)

        for ax in axes:
            ax.label_outer()